In [2]:
from typing import List
import re

import numpy as np
from matplotlib import pyplot as plt
import matplotlib

# include the path to the parent directory

import sys
sys.path.append('../')

from mfgpflow.data_loader import StellarMassFunctions

# set a random number seed to reproducibility
np.random.seed(0)

# Calibrate CAMELS Hydro-sim-code-calibration Emulator

Here we try to find the mapping from one simulation code to another, on the summary statistics level.

Let's try the matter power spectrum level first. And then move to Bispectrum.

- AR1: Linear multi-fidelity emulator from [Kennedy M., O’Hagan A., 2000, ](https://academic.oup.com/biomet/article-pdf/87/1/1/590577/870001.pdf)
- NARGP: Non-linear multi-fidelity emulator from Perdikaris P., Raissi M., Damianou A., Lawrence N. D., Karniadakis G. E., 2017, Proc. R. Soc. A., 473

- 50LR-3HR emulator: an emulator trained on 50 LR simulations and 3 HR simulations.

Data from Public Release of CAMELS.


In [5]:
def generate_data(folder: str = "../data/illustris/illustris-5-8/1004_LR_7_HR_test0/"):
    data = StellarMassFunctions(folder=folder)
    return data

base_folder = "../data/illustris/illustris-5-8-log1p/1004_LR_7_HR_test4/"
data = generate_data(base_folder)

In [ ]:
log10_mass_bins = np.array(
            [8.15, 8.45, 8.75, 9.05, 9.35, 9.65, 9.95, 10.25, 10.55, 10.85]
        )

In [ ]:
# visualize training spectra
for i, y_train in enumerate(data.Y_train[0]):
    if i == 0:
        plt.plot(y_train, label="low-fidelity", color="C0", ls="--", alpha=0.3)
    else:
        plt.plot(y_train, color="C0", ls="--", alpha=0.3)
for i, y_train in enumerate(data.Y_train[1]):
    if i == 0:
        plt.plot(y_train, label="high-fidelity", color="C1")
    else:
        plt.plot(y_train, color="C1")
plt.legend()
# plt.xlabel(r"$k (h/\mathrm{Mpc})$")
plt.ylabel(r"$SMF$")

(1004, 6)